# Graph Search Agent

The graph search agent requires an environment to define the following

1. Start State
2. Goal State
3. Possible Actions

We have to make a generalised agent, which reaches the goal state using the functions of the environment. Our agent will use BFS/DFS to reach to the goal state

In [3]:
import numpy as np

## Node
The Node class greates the graph node. It has the following values
1. Parent Node
2. State 
3. Cost

It makes use of the following built in functions: 
1. \_\_hash\_\_ : This provides the hash value for every node, which is required for the hashset
2. \_\_eq\_\_ : To check if 2 nodes are equal (Operator overload)
3. \_\_ne\_\_ : To check if 2 nodes are not equal (Operator overload)
4. \_\_str\_\_ : To get string representation of state in node

In [4]:
class Node:
    def __init__(self, parent, state, cost):
        
        self.parent = parent
        self.state = state
        self.cost = cost
    
    def __hash__(self):
        
        return hash(''.join(self.state.flatten()))
    
    def __str__(self):
        return str(self.state)
    
    def __eq__(self, other):
        
        return hash(''.join(self.state.flatten())) == hash(''.join(other.state.flatten())) 
    
    def __ne__(self, other):
        return hash(''.join(self.state.flatten())) != hash(''.join(other.state.flatten()))

## PriorityQueue
The Priority Queue is used to store the nodes along with the cost, and pop the node having the least cost for BFS

It makes use of the following functions: 
1. push : Add node to queue
2. pop : Pop node having least cost
3. is_empty : To check if queue is empty
4. \_\_len\_\_ : To get length of queue
5. \_\_str\_\_ : To get string representation of queue

In [5]:
class PriorityQueue():
    
    def __init__(self):
        self.queue = []
        
    def push(self, node):
        self.queue.append(node)
    
    def pop(self):
        
        next_state = None
        state_cost = 10**18
        index = -1
        
        for i in range(len(self.queue)):
            
            if self.queue[i].cost<state_cost:
                state_cost = self.queue[i].cost
                index = i
        
        return self.queue.pop(index)
    
    def is_empty(self):
        
        return len(self.queue)==0
    
    def __str__(self):
        l = []
        for i in self.queue:
            l.append(i.state)
        
        return str(l)
    
    def __len__(self):
        return len(self.queue)
            

## Environment

The environment is what the agent plays in. It has the following entities:
1. actions : The actions defined in the environment
2. start_state : The starting state of the environment
3. goal_state : The goal state of the environment

It has the following functions: 
1. get_start_state : returns the start state
2. reached_goal : returns goal_state
3. get_next_states : Given current state, it returns all possible next states

In [6]:
class Environment():
    
    def __init__(self, start_state = None, goal_state = None):
        self.actions = [1,2,3,4] #1 - Up, 2 - Down, 3 - Right, 4 - Left
        self.start_state = start_state
        self.goal_state = goal_state
    
    def get_start_state(self):
        return self.start_state
    
    def get_next_states(self, state):
        
        space = (0,0)
        for i in range(3):
            for j in range(3):
                if state[i,j] == '_':
                    space = (i,j)
                    break
        
        new_states = []
        
        if space[0] > 0:# Move Up
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]]  = new_state[space[0]-1, space[1]]
            new_state[space[0]-1, space[1]] = val
            
            new_states.append(new_state)
            
        if space[0] < 2: #Move down
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]]  = new_state[space[0]+1, space[1]]
            new_state[space[0]+1, space[1]] = val
            
            new_states.append(new_state)
        
        if space[1]<2: #Move right
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0], space[1]+1]
            new_state[space[0], space[1]+1] = val
            
            new_states.append(new_state)
            
        if space[1] > 0: #Move Left
            new_state = np.copy(state)
            
            val = new_state[space[0], space[1]]
            new_state[space[0], space[1]] = new_state[space[0], space[1]-1]
            new_state[space[0], space[1]-1] = val
            
            new_states.append(new_state)
        
        return new_states
    
    def reached_goal(self, state):
        
        for i in range(3):
            for j in range(3):
                if state[i,j] != self.goal_state[i,j]:
                    return False
        
        return True

Since the environment cannot spawn its own start and end states, we input the start state and end state to the environment

In [7]:
start_state = np.array([['_',2,4], [1,3,5], [7,8,6]])
goal_state = np.array([[1,2,3], [8,'_',4], [7,6,5]])
env = Environment(start_state, goal_state)

In [8]:
print(start_state)
print(goal_state)

[['_' '2' '4']
 ['1' '3' '5']
 ['7' '8' '6']]
[['1' '2' '3']
 ['8' '_' '4']
 ['7' '6' '5']]


### Explored
explored is a dictionary which stores all explored nodes.

### Frontier
Frontier is the queue which stores the next states for BFS

In [9]:
explored = dict()
frontier = PriorityQueue()

In [10]:
init_state = env.get_start_state()
init_node = Node(parent = None, state = init_state, cost = 0)
frontier.push(init_node)

In [11]:
goal_node = None
while not frontier.is_empty():
    
    curr_node = frontier.pop()
    next_states = env.get_next_states(curr_node.state)
    
    if hash(curr_node) in explored:
        continue
        
    explored[hash(curr_node)] = curr_node
    
    if env.reached_goal(curr_node.state):
        goal_node = curr_node
        break
    
    for state in next_states:
        node = Node(parent=curr_node, state=state, cost=curr_node.cost+1)
        frontier.push(node)

    

In [12]:
node = goal_node
l = []
while node is not None:
    l.append(node)
    node = node.parent

step = 1
for node in l[::-1]:
    print("Step: ",step)
    print(node)
    step+=1
    

Step:  1
[['_' '2' '4']
 ['1' '3' '5']
 ['7' '8' '6']]
Step:  2
[['2' '_' '4']
 ['1' '3' '5']
 ['7' '8' '6']]
Step:  3
[['2' '3' '4']
 ['1' '_' '5']
 ['7' '8' '6']]
Step:  4
[['2' '3' '4']
 ['1' '8' '5']
 ['7' '_' '6']]
Step:  5
[['2' '3' '4']
 ['1' '8' '5']
 ['7' '6' '_']]
Step:  6
[['2' '3' '4']
 ['1' '8' '_']
 ['7' '6' '5']]
Step:  7
[['2' '3' '_']
 ['1' '8' '4']
 ['7' '6' '5']]
Step:  8
[['2' '_' '3']
 ['1' '8' '4']
 ['7' '6' '5']]
Step:  9
[['_' '2' '3']
 ['1' '8' '4']
 ['7' '6' '5']]
Step:  10
[['1' '2' '3']
 ['_' '8' '4']
 ['7' '6' '5']]
Step:  11
[['1' '2' '3']
 ['8' '_' '4']
 ['7' '6' '5']]


In [25]:
len(frontier)

51122

In [26]:
len(explored)

72841

In [13]:
''.join(init_state.flatten())

'_24135786'